# Transform Tensorboard data to pandas DataFrame all at once, by Colab.

- [TensorBoard.dev PREVIEW
Easily host, track, and share your ML experiments for free.](https://tensorboard.dev/)

- [Accessing TensorBoard Data as DataFrames](https://www.tensorflow.org/tensorboard/dataframe_api)


In [1]:
FILE_NAME = 'model-relu'

## 1. (Pre-requisit) Upload data to 'logs' directory in Colab

In [2]:
ls

logs/           model-relu-loss-accuracy.csv  sample_data/
model-relu.csv  model-relu-win_ratio.csv


In [3]:
ls logs

events.out.tfevents.1614856791.tanaka-dynabook-R734-38KW.29576.5.v2


2. Upload data to Tensorboard.dev

In [4]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [5]:
!tensorboard dev upload --logdir ./logs/ \
  --name "model-sigmoid" \
  --description "use sigmoid at final activation of generator"\
  --one_shot

2021-03-06 05:34:24.476254: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Data for the "text" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/YaGXEaluRh2QzSRuy0KAEA/

[2021-03-06T05:34:26] Started scanning logdir.
[2021-03-06T05:34:42] Total uploaded: 16848 scalars, 0 tensors, 0 binary objects
[2021-03-06T05:34:42] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/YaGXEaluRh2QzSRuy0KAEA/


In [6]:
!tensorboard dev list

2021-03-06 05:34:44.191388: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Data for the "text" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.
https://tensorboard.dev/experiment/YaGXEaluRh2QzSRuy0KAEA/
	Name                 model-sigmoid
	Description          use sigmoid at final activation of generator
	Id                   YaGXEaluRh2QzSRuy0KAEA
	Created              2021-03-06 05:34:26 (23 seconds ago)
	Updated              2021-03-06 05:34:42 (7 seconds ago)
	Runs                 1
	Tags                 48
	Scalars              16845
	Tensor bytes         0
	Binary object bytes  0
https://tensorboard.dev/experiment/i7xCelHrTACDz5ocYE0Mdg/
	Name                 model-sigmoid
	Description          use sigmoid at final activation of generator
	Id                   i7xCelHrTACDz5ocYE0Mdg
	Creat

# 3. Download the data from tensorboard.dev

In [7]:
from packaging import version

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
import tensorboard as tb

In [8]:
major_ver, minor_ver, _ = version.parse(tb.__version__).release
assert major_ver >= 2 and minor_ver >= 3, \
    "This notebook requires TensorBoard 2.3 or later."
print("TensorBoard version: ", tb.__version__)

TensorBoard version:  2.4.1


##4. Transform to pandas DataFrame

###4.1 Copy Id from tensorboard dev list, then paset it to the following 'experiment_id.

In [9]:
experiment_id = "KXTKGGQTQoiQSMQ96uq2KQ"
experiment = tb.data.experimental.ExperimentFromDev(experiment_id)
df = experiment.get_scalars()
df

,run,tag,step,value
0,.,Evaluation: Fail ratio/whole [%],0,99.557999
1,.,Evaluation: Fail ratio/whole [%],10,89.916000
2,.,Evaluation: Fail ratio/whole [%],20,88.398003
3,.,Evaluation: Fail ratio/whole [%],30,87.926003
4,.,Evaluation: Fail ratio/whole [%],40,85.024002
...,...,...,...,...
16840,.,gan_loss,1997,133.796204
16841,.,gan_loss,1998,133.999603
16842,.,gan_loss,1999,133.809647
16843,.,gan_loss,2000,133.833542


##4.2 Reshape the DataFrame

In [10]:
#dfc=df.drop('step', axis=1)

In [11]:
#dfc

In [12]:
#print(df["tag"].unique())

In [13]:
dfc=pd.DataFrame()
dfc=df.set_index('step')
dfc = dfc.pivot(columns='tag', values='value')
dfc

tag,Evaluation: Fail ratio/whole [%],Evaluation: Success ratio/potential_win [%],Evaluation: Success ratio/whole [%],Evaluation: l1_blue_not_win/blue_not_win [%],Evaluation: l1_blue_not_win/whole [%],Evaluation: l2_blue_not_win/blue_not_win [%],Evaluation: l2_blue_not_win/whole [%],Evaluation: w1_blue_not_win/blue_not_win [%],Evaluation: w1_blue_not_win/whole [%],Evaluation: w1_blue_win/blue_win [%],Evaluation: w1_blue_win/whole [%],Evaluation: w2_blue_not_win/blue_not_win [%],Evaluation: w2_blue_not_win/whole [%],Evaluation: w2_blue_win/blue_win [%],Evaluation: w2_blue_win/whole [%],Evaluation: w3_blue_not_win/blue_not_win [%],Evaluation: w3_blue_not_win/whole [%],Evaluation: w3_blue_win/blue_win [%],Evaluation: w3_blue_win/whole [%],Evaluation: win_w1/potential_win [%],Evaluation: win_w2/potential_win [%],Evaluation: win_w3/potential_win [%],Test: Fail ratio/whole [%],Test: Success ratio/potential_win [%],Test: Success ratio/whole [%],Test: l1_blue_not_win/blue_not_win [%],Test: l1_blue_not_win/whole [%],Test: l2_blue_not_win/blue_not_win [%],Test: l2_blue_not_win/whole [%],Test: w1_blue_not_win/blue_not_win [%],Test: w1_blue_not_win/whole [%],Test: w1_blue_win/blue_win [%],Test: w1_blue_win/whole [%],Test: w2_blue_not_win/blue_not_win [%],Test: w2_blue_not_win/whole [%],Test: w2_blue_win/blue_win [%],Test: w2_blue_win/whole [%],Test: w3_blue_not_win/blue_not_win [%],Test: w3_blue_not_win/whole [%],Test: w3_blue_win/blue_win [%],Test: w3_blue_win/whole [%],Test: win_w1/potential_win [%],Test: win_w2/potential_win [%],Test: win_w3/potential_win [%],discriminator_accuracy,discriminator_loss,gan_accuracy,gan_loss
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,99.557999,0.753110,0.442000,21.978144,21.881001,19.515257,19.429001,3.207176,3.193,0.0,0.0,8.505594,8.468,0.000000,0.000,46.793827,46.587002,100.000000,0.442000,0.0,0.000000,0.753110,94.561996,9.309412,5.438,23.251411,21.987,20.726084,19.599001,3.354413,3.172,0.000000,0.00,8.926418,8.441,0.0,0.0,43.741673,41.362999,100.000000,5.438000,0.000000,0.0,9.309412,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.025391,138.690460,81.417969,148.284042
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139.220703,132.955811,81.039062,150.465332
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137.062500,131.036758,90.273438,149.404175
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,129.404297,132.851959,98.765625,147.460205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.722656,142.379227,179.339844,133.796204
1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.660156,142.388382,178.312500,133.999603
1999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.728516,142.370209,179.332031,133.809647


###4.3 Make DataFrame for Win/Lose ratio

In [14]:
dfw = dfc.drop(columns=dfc.columns[[-1,-2,-3,-4]])
dfw = dfw.dropna(how='all')
dfw

tag,Evaluation: Fail ratio/whole [%],Evaluation: Success ratio/potential_win [%],Evaluation: Success ratio/whole [%],Evaluation: l1_blue_not_win/blue_not_win [%],Evaluation: l1_blue_not_win/whole [%],Evaluation: l2_blue_not_win/blue_not_win [%],Evaluation: l2_blue_not_win/whole [%],Evaluation: w1_blue_not_win/blue_not_win [%],Evaluation: w1_blue_not_win/whole [%],Evaluation: w1_blue_win/blue_win [%],Evaluation: w1_blue_win/whole [%],Evaluation: w2_blue_not_win/blue_not_win [%],Evaluation: w2_blue_not_win/whole [%],Evaluation: w2_blue_win/blue_win [%],Evaluation: w2_blue_win/whole [%],Evaluation: w3_blue_not_win/blue_not_win [%],Evaluation: w3_blue_not_win/whole [%],Evaluation: w3_blue_win/blue_win [%],Evaluation: w3_blue_win/whole [%],Evaluation: win_w1/potential_win [%],Evaluation: win_w2/potential_win [%],Evaluation: win_w3/potential_win [%],Test: Fail ratio/whole [%],Test: Success ratio/potential_win [%],Test: Success ratio/whole [%],Test: l1_blue_not_win/blue_not_win [%],Test: l1_blue_not_win/whole [%],Test: l2_blue_not_win/blue_not_win [%],Test: l2_blue_not_win/whole [%],Test: w1_blue_not_win/blue_not_win [%],Test: w1_blue_not_win/whole [%],Test: w1_blue_win/blue_win [%],Test: w1_blue_win/whole [%],Test: w2_blue_not_win/blue_not_win [%],Test: w2_blue_not_win/whole [%],Test: w2_blue_win/blue_win [%],Test: w2_blue_win/whole [%],Test: w3_blue_not_win/blue_not_win [%],Test: w3_blue_not_win/whole [%],Test: w3_blue_win/blue_win [%],Test: w3_blue_win/whole [%],Test: win_w1/potential_win [%],Test: win_w2/potential_win [%],Test: win_w3/potential_win [%]
step,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,99.557999,0.753110,0.442000,21.978144,21.881001,19.515257,19.429001,3.207176,3.193,0.000000,0.000,8.505594,8.468,0.000000,0.000,46.793827,46.587002,100.000000,0.442000,0.000000,0.000000,0.753110,94.561996,9.309412,5.438000,23.251411,21.987,20.726084,19.599001,3.354413,3.172,0.000000,0.000,8.926418,8.441,0.000000,0.000,43.741673,41.362999,100.000000,5.438000,0.000000,0.000000,9.309412
10,89.916000,17.181803,10.084000,24.334934,21.881001,21.607944,19.429001,3.393167,3.051,1.408171,0.142,9.388762,8.442,0.257834,0.026,41.275188,37.112999,98.333992,9.916000,0.241949,0.044301,16.895554,91.786003,14.061698,8.214000,23.954634,21.987,21.352930,19.599001,3.249951,2.983,2.300950,0.189,9.146275,8.395,0.560019,0.046,42.296211,38.821999,97.139030,7.979000,0.323553,0.078748,13.659397
20,88.398003,19.768274,11.602000,24.752823,21.881001,21.979004,19.429001,3.414104,3.018,1.508361,0.175,9.502478,8.400,0.586106,0.068,40.351593,35.669998,97.905533,11.359000,0.298177,0.115863,19.354235,100.000000,0.000000,0.000000,21.987000,21.987,19.599001,19.599001,3.172000,3.172,NaN,0.000,8.441000,8.441,NaN,0.000,46.800999,46.800999,NaN,0.000000,0.000000,0.000000,0.000000
30,87.926003,20.572500,12.074000,24.885700,21.881001,22.096991,19.429001,3.417647,3.005,1.557065,0.188,9.621727,8.460,0.066258,0.008,39.977936,35.151001,98.376678,11.878000,0.320327,0.013631,20.238541,88.265999,20.087650,11.734000,24.909931,21.987,22.204473,19.599001,3.464528,3.058,0.971536,0.114,9.142818,8.070,3.161752,0.371,40.278252,35.551998,95.866714,11.249000,0.195159,0.635122,19.257370
40,85.024002,25.517124,14.976000,25.735086,21.881001,22.851194,19.429001,3.477842,2.957,1.575855,0.236,9.947780,8.458,0.066774,0.010,37.988098,32.299000,98.357368,14.730000,0.402113,0.017039,25.097973,87.699997,21.056597,12.300000,25.070696,21.987,22.347776,19.599001,3.337514,2.927,1.991870,0.245,9.591790,8.412,0.235772,0.029,39.652222,34.775002,97.772354,12.026000,0.419420,0.049646,20.587530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1960,63.957001,61.412506,36.042999,34.212048,21.881001,30.378223,19.429001,4.992417,3.193,0.000000,0.000,9.248402,5.915,7.083206,2.553,21.168911,13.539000,92.916794,33.490002,0.000000,4.349975,57.062531,54.147999,78.494881,45.852001,40.605377,2

###4.4 Make DataFrame for Loss/Accuracy

In [15]:
dfg = dfc[['discriminator_accuracy', 'discriminator_loss', 'gan_accuracy','gan_loss']]

In [16]:
dfg=dfg.dropna(how='all')

In [17]:
dfg

tag,discriminator_accuracy,discriminator_loss,gan_accuracy,gan_loss
step,,,,
1,122.025391,138.690460,81.417969,148.284042
2,139.220703,132.955811,81.039062,150.465332
3,137.062500,131.036758,90.273438,149.404175
4,129.404297,132.851959,98.765625,147.460205
5,121.009766,136.707840,108.917969,142.814606
...,...,...,...,...
1997,102.722656,142.379227,179.339844,133.796204
1998,102.660156,142.388382,178.312500,133.999603
1999,102.728516,142.370209,179.332031,133.809647


##5. Save DataFrame

In [18]:
dfw.to_csv(FILE_NAME + '-win_ratio' '.csv')
dfg.to_csv(FILE_NAME + '-loss-accuracy' '.csv')

##6. Don't forget down load the data from colab